# Scraper

In [3]:
import bs4
import urllib
import numpy as np
import string

## Le Monde main page

Get the main page in order to get all the article links. 

In [4]:
lemonde = bs4.BeautifulSoup( urllib.request.urlopen("http://www.lemonde.fr/").read(), "lxml")

In [5]:
links = []
for a in lemonde('a'):
    links.append(a['href'])
links = np.array(links)

In [6]:
articles = []
for l in links:
    if "article" in l:
        articles.append( l )
articles = np.array(articles)
articles.size

182

## Article example

Get an article and extract the useful information.

In [27]:
lemonde_article_url = "http://www.lemonde.fr/" + articles[0]
article_lemonde = bs4.BeautifulSoup( urllib.request.urlopen(lemonde_article_url).read(), "lxml" )
print( "Article: %s" % lemonde_article_url)
len(article_lemonde.text)

Article: http://www.lemonde.fr//europe/article/2017/11/03/carles-puigdemont-veut-se-presenter-aux-elections-de-decembre_5209996_3214.html


33355

### Useful information

After a study of what is displayed on an article, I chose to extract those elements:
 - Title
 - Url
     - Category 1
     - Category 2
 - Writer
 - Article description
 - Article summary
 - Publish Time
 - Update Time
 - Article
     - Titles
     - Text
 - Related article titles
 
Those elements are not loaded with the main static HTML. It is necessary to make supplementary requests or/and execute javascript.
In a first time, I won't scrape them. As, there is already enough to practice with the previous elements.
 - Comments
     - Writer
     - Date
     - Content
 - \# facebook shares

In [26]:
# Title
title = article_lemonde('h1', {'class': 'tt2'})[0].string
print ( "* Title: %s" % title)

# Categories
slash1 = articles[0].index('/')+1
slash2 = articles[0].index('/', slash1)+1
slash3 = articles[0].index('/', slash2)
category_1 = articles[0][slash1:slash2-1]
category_2 = articles[0][slash2:slash3]
print ( "* Category 1: %s" % category_1)
print ( "* Category 2: %s" % category_2)

# Writer
writer = article_lemonde('span', {'id': 'publisher'})[0].string
print ( "* Writer: %s" % writer)

# Date
publish_time = article_lemonde('time', {'itemprop': 'datePublished'})[0]['datetime']
print ( "* Publish time: %s" % publish_time)
update_time = article_lemonde('time', {'itemprop': 'dateModified'})[0]['datetime']
print ( "* Update time: %s" % update_time)

# Short

print()
article_description = article_lemonde('', {'class':'description-article'})[0].string
article_summary = article_lemonde('h2', {'class':'taille_courante'})[0].text
print ( "* Description: \n%s" % article_description)
print ( "* Summary: \n%s" % article_summary)

# Complete
print()
article_paragraphs = []
for paragraph in article_lemonde('div', {'id':'articleBody'})[0]('p'):
    if paragraph.get('class') is None:
        article_paragraphs.append(paragraph.text)
print ( "* First paragraph: \n%s" % article_paragraphs[0])

article_intertitres = []
for inter in article_lemonde('h2', {'class':'intertitre'}):
    article_intertitres.append(inter.text)
print ( "* First inter title: \n%s" % article_intertitres[0])

related_articles = []
for art in article_lemonde('aside', {'class':'bloc_base meme_sujet'})[0]('a'):
    related_articles.append(art.text)
print ( "* First related article title: \n%s" % related_articles[0].strip())

* Title: La justice espagnole lance un mandat d’arrêt contre Carles Puigdemont
* Category 1: europe
* Category 2: article
* Writer: Le Monde
* Publish time: 2017-11-03T20:07:00+01:00
* Update time: 2017-11-03T21:10:44+01:00

* Description: 
Le président de l’exécutif catalan déchu a dit, vendredi, qu’il n’avait pas l’intention de fuir la justice et qu’il entendait se présenter aux prochaines élections.
* Summary: 
La justice espagnole a lancé, vendredi 3 novembre, un mandat d’arrêt européen contre le président catalan destitué Carles Puigdemont et contre quatre de ses « ministres » repliés en Belgique, qui ont refusé de comparaître devant elle.

* First paragraph: 
Une juge d’instruction a lancé le mandat contre chacun des cinq membres du gouvernement destitué, pour « rébellion, sédition, détournement de fonds publics et désobéissance à l’autorité », a annoncé dans un communiqué le tribunal, qui a déjà incarcéré, jeudi, huit autres membres de l’ancien exécutif indépendantiste. Eric Van